In [5]:
# Enable auto-reloading of external modules - useful during development
%load_ext autoreload
%autoreload 2

# Configure Python path to find our custom modules
import sys
from pathlib import Path

# Add project root to the Python path for proper imports
project_root = Path.cwd().parent
if project_root not in sys.path:
    sys.path.insert(0, str(project_root))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Import necessary libraries
import src.processing as processing
import src.config as lists

In [7]:
# Load and process data
df = processing.load_data("/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv")
df_prepared = processing.prepare_data(df)
df_added_features = processing.create_all_model_features_orchestrated(df_prepared)
df_missing = processing.drop_missing_final_vars_streamlined(df_added_features, lists.final_set_A_predictor_names_and_dependent)
df_final = processing.annual_winsorize_variables(df_missing, lists.columns_to_winsorize)

/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/src/processing.py:42: DtypeWarning: Columns (10,11,12,16,17,22,26,30,33,34,35,945,946,948,949,950,955,969,970,975,980) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Data loaded successfully from /Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv
Original number of observations: 317304
Number of columns after selection: 30
Observations after year filter (2000-2023): 302751
Observations after excluding financial and utility firms: 170598
Starting feature construction. Initial df shape: (170598, 30)
  Creating lags for: ['at', 'ni', 'rect', 'invt', 'ap', 'sale']

Performing pre-calculation validity checks & preparations...
  Missing 'xrd' values filled with 0.
  'ipo_year' created from 'ipodate'.

Constructing dependent variable...
  OCF_Scaled_t_plus_1 created.

Constructing Set A (OLS) predictors...
  Set A predictors constructed.

Constructing control dummy variables...
  Dummy variables constructed.

Constructing Set B (additional ML) predictors...
  Set B predictors constructed.

Selecting final model variables and dropping intermediate columns...
  Shape of DataFrame 

In [8]:
# Split data chronologically - train on pre-2018, test on 2018+
train_df, test_df = processing.split_data_chronologically(df_final, 'fyear', split_year=2018)

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Training period: {train_df['fyear'].min()} - {train_df['fyear'].max()}")
print(f"Test period: {test_df['fyear'].min()} - {test_df['fyear'].max()}")

Training set: 102567 obs (Predictor years <= 2018)
Test set: 20882 obs (Predictor years > 2018)
Training data shape: (102567, 31)
Test data shape: (20882, 31)
Training period: 2001.0 - 2018.0
Test period: 2019.0 - 2022.0


In [9]:
# Set B (Additional ML) predictor feature names
SET_B_FEATURES = [
    'XSGA_Scaled_t', 'XRD_Scaled_t', 'CAPX_Scaled_t', 'CurrentRatio_t',
    'DebtToAssets_t', 'OCFtoSales_t', 'InvTurnover_t', 'RecTurnover_t',
    'GPM_t', 'Delta_Sales_Scaled_t', 'NI_Scaled_Lag_t',
    'CapitalIntensity_t', 'MkBk_t', 'FirmAge_t'
]

# Prepare features and dependent variable for XGBoost regression
# Using SET_A + SET_B + dummy controls to show the full power of ML
X_train = train_df[lists.SET_A_FEATURES + lists.SET_B_FEATURES + lists.OLS_DUMMY_FEATURES_FINAL]
y_train = train_df[lists.DEPENDENT_VARIABLE]

X_test = test_df[lists.SET_A_FEATURES + lists.SET_B_FEATURES + lists.OLS_DUMMY_FEATURES_FINAL]
y_test = test_df[lists.DEPENDENT_VARIABLE]

print("Features included in the model:")
for i, feature in enumerate(X_train.columns, 1):
    print(f"{i:2d}. {feature}")

print(f"\nDependent variable: {lists.DEPENDENT_VARIABLE}")
print(f"Training observations: {len(X_train)}")
print(f"Test observations: {len(X_test)}")

Features included in the model:
 1. OCF_Scaled_Lag_t
 2. NI_Scaled_t
 3. Accruals_Scaled_t
 4. Delta_Rec_Scaled_t
 5. Delta_Inv_Scaled_t
 6. Delta_AP_Scaled_t
 7. DP_Scaled_t
 8. ln_at_t
 9. XSGA_Scaled_t
10. XRD_Scaled_t
11. CAPX_Scaled_t
12. CurrentRatio_t
13. DebtToAssets_t
14. OCFtoSales_t
15. InvTurnover_t
16. RecTurnover_t
17. GPM_t
18. Delta_Sales_Scaled_t
19. NI_Scaled_Lag_t
20. CapitalIntensity_t
21. MkBk_t
22. FirmAge_t
23. ASC606_TCJA_combined_dummy

Dependent variable: OCF_Scaled_t_plus_1
Training observations: 102567
Test observations: 20882


In [10]:
# =============================================================================
# BAYESIAN OPTIMIZATION FOR HYPERPARAMETER TUNING
# =============================================================================

import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import time

# Define hyperparameter search space for Bayesian optimization
search_space = {
    'max_depth': Integer(3, 10),
    'learning_rate': Real(0.01, 0.2, prior='log-uniform'),
    'n_estimators': Integer(100, 300),
    'subsample': Real(0.8, 1.0),
    'colsample_bytree': Real(0.8, 1.0)
}

# Create XGBoost regressor
xgb_regressor = xgb.XGBRegressor(random_state=42, objective='reg:squarederror')

# Perform Bayesian Search with 5-fold cross-validation
print("Performing 5-fold cross-validation for XGBoost Bayesian hyperparameter tuning...")
print(f"Search space dimensions: {len(search_space)}")
print(f"Number of iterations: 100 (intelligent sampling)")

start_time = time.time()

bayes_search = BayesSearchCV(
    estimator=xgb_regressor,
    search_spaces=search_space,
    n_iter=100,  # Number of parameter settings to try
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42,
    verbose=2  # Increased verbosity for more detailed progress information
)

# Fit the bayesian search
bayes_search.fit(X_train, y_train)

# Print best parameters and score
print(f"\nBest hyperparameters: {bayes_search.best_params_}")
print(f"Best cross-validation R² score: {bayes_search.best_score_:.4f}")
print(f"Standard deviation: {bayes_search.cv_results_['std_test_score'][bayes_search.best_index_]:.4f}")

elapsed_time = time.time() - start_time
print(f"Bayesian search completed in {elapsed_time:.2f} seconds")

# For compatibility with rest of the code, assign to grid_search variable
grid_search = bayes_search

Performing 5-fold cross-validation for XGBoost Bayesian hyperparameter tuning...
Search space dimensions: 5
Number of iterations: 100 (intelligent sampling)
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.9340295896537869; total time=   8.6s
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.9340295896537869; total time=   8.8s
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.9340295896537869; total time=   8.9s
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.9340295896537869; total time=   8.9s
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.93402958965

In [11]:
# =============================================================================
# XGBOOST MODEL - For ML Model Comparisons
# =============================================================================

# Use the best model from grid search
best_xgb_model = grid_search.best_estimator_

# Predictions
y_train_pred = best_xgb_model.predict(X_train)
y_test_pred = best_xgb_model.predict(X_test)

# Simple metrics function for any model
def print_model_performance(y_true, y_pred, model_name, dataset):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{model_name} ({dataset}): R²={r2:.4f} | RMSE={rmse:.4f} | MAE={mae:.4f}")

# Print results
print("XGBOOST PERFORMANCE (SET A + SET B FEATURES):")
print_model_performance(y_train, y_train_pred, "XGBoost", "Train")
print_model_performance(y_test, y_test_pred, "XGBoost", "Test")

XGBOOST PERFORMANCE (SET A + SET B FEATURES):
XGBoost (Train): R²=0.8162 | RMSE=0.4544 | MAE=0.1564
XGBoost (Test): R²=0.6871 | RMSE=0.5456 | MAE=0.1866


In [12]:
# =============================================================================
# FEATURE IMPORTANCE ANALYSIS
# =============================================================================

import pandas as pd

# Get feature importances
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("FEATURE IMPORTANCE RANKING (SET A + SET B FEATURES):")
print("="*50)
for i, (_, row) in enumerate(feature_importance.iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<25} {row['importance']:.4f}")

# Print model details
print(f"\nMODEL DETAILS:")
print(f"Max Depth: {best_xgb_model.max_depth}")
print(f"Learning Rate: {best_xgb_model.learning_rate}")
print(f"N Estimators: {best_xgb_model.n_estimators}")
print(f"Subsample: {best_xgb_model.subsample}")
print(f"Colsample Bytree: {best_xgb_model.colsample_bytree}")
print(f"Number of Features: {best_xgb_model.n_features_in_}")

FEATURE IMPORTANCE RANKING (SET A + SET B FEATURES):
 1. NI_Scaled_t               0.3751
 2. XSGA_Scaled_t             0.1489
 3. ln_at_t                   0.0768
 4. OCF_Scaled_Lag_t          0.0763
 5. OCFtoSales_t              0.0336
 6. DebtToAssets_t            0.0304
 7. ASC606_TCJA_combined_dummy 0.0297
 8. XRD_Scaled_t              0.0256
 9. MkBk_t                    0.0186
10. Accruals_Scaled_t         0.0178
11. Delta_AP_Scaled_t         0.0174
12. InvTurnover_t             0.0161
13. NI_Scaled_Lag_t           0.0156
14. CapitalIntensity_t        0.0132
15. RecTurnover_t             0.0131
16. CurrentRatio_t            0.0131
17. FirmAge_t                 0.0127
18. GPM_t                     0.0123
19. Delta_Sales_Scaled_t      0.0119
20. Delta_Rec_Scaled_t        0.0117
21. DP_Scaled_t               0.0114
22. CAPX_Scaled_t             0.0093
23. Delta_Inv_Scaled_t        0.0092

MODEL DETAILS:
Max Depth: 5
Learning Rate: 0.02520925149700464
N Estimators: 300
Subsample: 0.